In [1]:
import os
execfile(os.path.join(os.environ["SPARK_HOME"], 'python/pyspark/shell.py'))

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 1.3.0
      /_/

Using Python version 2.7.6 (default, Mar 22 2014 22:59:56)
SparkContext available as sc, HiveContext available as sqlCtx.


In [2]:
import json

json_data = sc.textFile("/recsys/DO_record_per_line.json")
list_json = json_data.map(lambda x: json.loads(x))
list_json.first()

{u'cat': u'3/business_management|6/economics_finance',
 u'desc': u'This course introduces the basic financial statements used by most businesses, as well as the essential tools used to prepare them. This course will serve as a resource to help business students succeed in their upcoming university-level accounting classes, and as a refresher for upper division accounting students who are struggling to recall elementary concepts essential to more advanced accounting topics. Business owners will also benefit from this class by gaining essential skills necessary to organize and manage information pertinent to operating their business. At the conclusion of the class, students will understand the balance sheet, income statement, and cash flow statement. They will be able to differentiate between cash basis and accrual basis techniques, and know when each is appropriate. They\u2019ll also understand the accounting equation, how to journalize and post transactions, how to adjust and close acc

In [3]:
df_json = sqlCtx.jsonFile("/recsys/DO_record_per_line.json")
df_json.show()

cat                  desc                 id lang name                 provider      
3/business_manage... This course intro... 4  en   Accounting Cycle:... Canvas Network
11/law               This online cours... 5  en   American Counter ... Canvas Network
5/computer_scienc... This course is ta... 6  fr   Arithmétique: en ... Canvas Network
14/social_sciences   We live in a digi... 7  en   Becoming a Dynami... Canvas Network
2/biology_life_sc... This self-paced c... 8  en   Bioethics            Canvas Network
9/humanities|15/m... This game-based c... 9  en   College Foundatio... Canvas Network
14/social_sciences   What’s in your di... 10 en   Digital Literacies I Canvas Network
14/social_sciences   The goal of the D... 11 en   Digital Literacie... Canvas Network
14/social_sciences   Ready to explore ... 12 en   Digital Tools for... Canvas Network
14/social_sciences   This self-paced c... 13 en   Discover Your Val... Canvas Network
12/medicine_health   What is “interpro... 14 en   Enha

In [36]:
import nltk
from nltk.stem.snowball import SnowballStemmer
""""
from nltk.stem.api import StemmerI
from nltk.stem.regexp import RegexpStemmer
from nltk.stem.lancaster import LancasterStemmer
from nltk.stem.isri import ISRIStemmer
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.rslp import RSLPStemmer
"""
import re
from pymystem3 import Mystem
from bs4 import BeautifulSoup # удобная библиотека для обработки html-тегов, которые есть в текстах

m = Mystem()
#wnl = WordNetLemmatizer()
swEn = nltk.corpus.stopwords.words('english')
swRu = nltk.corpus.stopwords.words('russian')
swEs = nltk.corpus.stopwords.words('spanish')
reWords = re.compile(u'[áéíóúñüа-яёa-z\d]+')#\u00C0-\u00FF 

def getWords(text,lang="en",minLen=2):
    text = text.strip()
    if text > "":
        text = BeautifulSoup(text).get_text() # удаляем html-теги

        if lang == "ru":
            words = m.lemmatize(text)
            words = [token.lower() for token in words if any([c for c in token.strip() if c.isalpha()])]
        else:
            words = reWords.findall(text.lower())
            words = [token for token in words if any([c for c in token.strip() if c.isalpha()])]

        if minLen > 0:
            words = [word for word in words if len(word) >= minLen]

        if lang == 'en':
            stopWords = swEn
        elif lang == 'ru':
            stopWords = swRu + swEn
        elif lang == 'es':
            stopWords = swEs
        else:
            stopWords = []

        if len(stopWords) > 0: 
            words = [word for word in words if word not in stopWords]

        if lang == "en":
            #stemmer = PorterStemmer()
            #stemmer = LancasterStemmer()
            stemmer = SnowballStemmer("english")
            words = [stemmer.stem(t) for t in words]
            #words = [wnl.lemmatize(t, pos='v') for t in words]
            #words = [wnl.lemmatize(t) for t in words]
        elif lang == "es":
            stemmer = SnowballStemmer("spanish")
            words = [stemmer.stem(t) for t in words]
        """
        elif lang == "ru":
            stemmer = SnowballStemmer("russian")
            words = [stemmer.stem(t) for t in words]
        """
    else:
        words = ()
    return words

def getWords2(text,lang='en',minLen=2):
    text = BeautifulSoup(text).get_text()
    words = reWords.findall(text.lower())
    words = [token for token in words if any([c for c in token.strip() if c.isalpha()])]
    if minLen > 0:
        words = [word for word in words if len(word) >= minLen]
        
    if lang == 'en':
        stopWords = swEn
    elif lang == 'ru':
        stopWords = swRu + swEn
    elif lang == 'es':
        stopWords = swEs
    else:
        stopWords = []    
    if len(stopWords) > 0: 
        words = [word for word in words if word not in stopWords]
        
    return words

def getWords3(text):
    return reWords.findall(text.lower())

In [37]:
from pyspark.mllib.feature import HashingTF
from pyspark.mllib.feature import IDF
from pyspark.mllib.feature import Normalizer

def calc_cosine_similarity(a,b):
    return a.dot(b)

def fchosen_doc(idnum,df_rec=None):
    if df_rec == None:
        df_rec = df_json[df_json.id == idnum].first()
    return normalizer_l2.transform(
        idf_fit.transform(
            tf.transform(
                getWords(df_rec[4]+'. '+df_rec[1]+'. '+df_rec[0],df_rec[3]))))

normalizer_l2 = Normalizer(p=2)
tf, idf = HashingTF(), IDF()

tokens = df_json.map(lambda x: getWords(x.name+'. '+x.desc+'. '+x.cat,x.lang))
doc_tf = tf.transform(tokens)

idf_fit = idf.fit(doc_tf)
tfidf = idf_fit.transform(doc_tf)
normalized_tf_idf_vectors = normalizer_l2.transform(tfidf)

Py4JJavaError: An error occurred while calling o561.fitIDF.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 66.0 failed 4 times, most recent failure: Lost task 0.3 in stage 66.0 (TID 189, ip-172-31-41-155.eu-west-1.compute.internal): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/cloudera/parcels/CDH-5.4.0-1.cdh5.4.0.p0.27/jars/spark-assembly-1.3.0-cdh5.4.0-hadoop2.6.0-cdh5.4.0.jar/pyspark/worker.py", line 101, in main
    process()
  File "/opt/cloudera/parcels/CDH-5.4.0-1.cdh5.4.0.p0.27/jars/spark-assembly-1.3.0-cdh5.4.0-hadoop2.6.0-cdh5.4.0.jar/pyspark/worker.py", line 96, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/opt/cloudera/parcels/CDH-5.4.0-1.cdh5.4.0.p0.27/jars/spark-assembly-1.3.0-cdh5.4.0-hadoop2.6.0-cdh5.4.0.jar/pyspark/serializers.py", line 236, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "<ipython-input-37-512f5d458c05>", 

In [ ]:
ids = [8200]#14398, 24543, 12873, 13449, 8200, 1133]
top11 = list()

for idnum in ids:    
    df_rec = df_json[df_json.id == idnum].first()
    lang = df_rec[3]
    
    chosen_doc = fchosen_doc(idnum,df_rec)
    #chosen_doc_br = sc.broadcast(chosen_doc)
    similarities = normalized_tf_idf_vectors.map(lambda x: calc_cosine_similarity(x, chosen_doc))
    
    listid = df_json.map(lambda x: x.id).collect()
    listok = similarities.collect()
    ok = sorted(zip(listid,listok),key = lambda x: -x[1])[:11]
    
    #ok = df_json.map(lambda x: x.id).zip(similarities).sortBy(lambda x: -x[1]).take(11)

    top11.append((idnum,lang,ok))
    print idnum

In [25]:
top11

[]

In [42]:
words = getWords('')#df_json[df_json.id == 8200].first()[1],'ru')
words

()

In [33]:
import json

ok = dict()
for val in top11:
    s = str(val[0])
    ok[s] = map(lambda x: x[0],filter(lambda x: x[0] != val[0],val[2]))
out = json.dumps(ok, sort_keys=False)
print out
f = open(r"lab10.json","w")
f.write(out)
f.close()

{}


In [ ]:
for x in top11:
    print x[0],x[1]
    for y in x[2]:
        print '  ',y[0],',',y[1],':',df_json[df_json.id == y[0]].collect()[0][4]

In [15]:
print df_json[df_json.id == 12802].collect()[0][4]

Aprende a tocar el teclado como los expertos by Virtuosso Producciones


In [16]:
print df_json[df_json.id == 12658].collect()[0][4]

Aprende a tocar el piano desde cero by Virtuosso Producciones


In [20]:
{1,2} & {2,3}

{2}